In [1]:
import sys
import random 
import requests
from time import sleep

import numpy as np

# assumes working directory is notebook location
corepath = "../../../helao-core"
sys.path.append(corepath)
from helaocore.models.experiment import ExperimentTemplate

sys.path.append("../..")
from helao.helpers.premodels import Sequence
from helao.configs.simulate import config as global_cfg

cfg = global_cfg['servers']

In [3]:
global_cfg

{'dummy': True,
 'simulation': True,
 'experiment_libraries': ['simulate_exp'],
 'sequence_libraries': [],
 'run_type': 'simulation',
 'root': '/mnt/BIGSTOR/INST',
 'servers': {'ORCH': {'host': '127.0.0.1',
   'port': 8001,
   'group': 'orchestrator',
   'fast': 'async_orch2',
   'params': {'enable_op': True, 'bokeh_port': 5002}},
  'PAL': {'host': '127.0.0.1',
   'port': 8003,
   'group': 'action',
   'fast': 'archive_simulator',
   'params': {'data_path': '/mnt/k/users/guevarra/20191108_multipH_OER_full.csv'}},
  'MOTOR': {'host': '127.0.0.1',
   'port': 8004,
   'group': 'action',
   'fast': 'motion_simulator',
   'params': {'platemap_path': '/mnt/j/hte_jcap_app_proto/map/0069-04-0100-mp.txt',
    'count_to_mm': {'A': 0.00015632645340611895, 'B': 0.00015648717587593696},
    'def_speed_count_sec': 10000,
    'max_speed_count_sec': 25000}},
  'PSTAT': {'host': '127.0.0.1',
   'port': 8005,
   'group': 'action',
   'fast': 'pstat_simulator',
   'params': {'data_path': '/mnt/k/users/gu

# request explorable space

In [ ]:
# use robotic sampler 'PAL' action server as placeholder for sample database server
resp = requests.post(f"http://{cfg['PAL']['host']}:{cfg['PAL']['port']}/list_all_spaces")
resp.status_code == 200

In [ ]:
# available composition,pH spaces
sorted([(d['elements'], d['solution_ph']) for d in resp.json()])

### terminology
- a 'sequence' is a queue of experiments performed in order
- an 'experiment' is a queue of actions performed in order
- _side note:_ one or more ESAMP processes may be created from an experiment, the experiment `SIM_measure_CP` produces 1 process
- an 'action' is the atomic request dispatched by the Orchestrator to individual action servers
- an 'action server' exposes hardware driver and data management functions via FastAPI
- an 'Orchestrator' manages the queing and dispatch sequences, experiments, and action requests

In [2]:
# valid experiment names
import helao.experiments.simulate_exp
helao.experiments.simulate_exp.__all__

['SIM_measure_CP']

In [3]:
from helao.experiments.simulate_exp import SIM_measure_CP
SIM_measure_CP

<function helao.experiments.simulate_exp.SIM_measure_CP(experiment: helao.helpers.premodels.Experiment, experiment_version: int = 1, solution_ph: Optional[int] = 13, elements: Optional[List[str]] = [], element_fracs: Optional[List[float]] = [])>

- `SIM_measure_CP` is the only experiment available in this simulator
- `SIM_measure_CP` has 3 real arguments: `solution_ph`, `elements`, and `element_fracs`
- args `experiment` and `experiment_version` are managed by orchestrator

### `SIM_measure_CP` experiment performs 9 actions:
1. query available plates for elements (and pH) matching `solution_ph` and `elements`
2. load plate_id identified in (1)
3. query available samples for element fractions matching `element_fracs`
4. locate x,y coordinates for sample identified in (3)
5. move stage motors to x,y, coordinates identified in (4)
6. run CP measurement at 3 mA/cm2 for 15 seconds
7. extract Eta (V vs O2/H2O) from measurement in (6)
8. run CP measurement at 10 mA/cm2 for 15 seconds
9. extract Eta (V vs O2/H2O) from measurement in (9)

# example Ni-Fe-La-Ce-Co-Ta @ pH=13

In [ ]:
import json
json.dumps(['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta'])

In [ ]:
# get addressable composition space (X's) from previous request
elements = ['Ni', 'Fe', 'La', 'Ce', 'Co', 'Ta']
solution_ph = 13

comp_space = [x for x in resp.json() if x['elements']==elements and x['solution_ph']==solution_ph][0]['element_fracs']
len(comp_space)

In [ ]:
comp_space[:5]

In [ ]:
# initial random seed of 5 compositions
random.seed(0)
comp_inds = random.sample(range(len(comp_space)), 5)

# create sequence object for holding experiments
sequence = Sequence(sequence_name='seed_sequence')

# populate sequence's experiment list
for i in comp_inds:
    sequence.experiment_plan_list.append(
        ExperimentTemplate(
            experiment_name="SIM_measure_CP",
            experiment_params={
                "solution_ph": 13,
                "elements": ["Ni", "Fe", "La", "Ce", "Co", "Ta"],
                "element_fracs": comp_space[i],
            },
        )
    )

In [ ]:
# preview sequence object, identifying info such as uuid and timestamp are only created when a sequence is dispactched (executed by Orchestrator)
sequence.as_dict()

In [ ]:
# send sequence to Orchestrator
seq_req = requests.post(f"http://{cfg['ORCH']['host']}:{cfg['ORCH']['port']}/append_sequence", json={"sequence": sequence.as_dict()})
seq_req.status_code == 200  # successful post request

In [ ]:
# get list of loaded sequences on Orchestrator
orch_list = requests.post(f"http://{cfg['ORCH']['host']}:{cfg['ORCH']['port']}/list_sequences")
orch_list.status_code == 200  # successful post request

In [ ]:
orch_list.json() # present sequence queue

In [ ]:
# start Orch (begin or resume dispatching sequence/experiment/action queues)
orch_start = requests.post(f"http://{cfg['ORCH']['host']}:{cfg['ORCH']['port']}/start")
orch_start.status_code == 200

__Notes on Orch status:__

The orchestrator server holds minimal state variables, so we can only ask:
1. whether it's currently stopped or running
2. the state of dispatched actions of the active experiment

When Orch completes all queued actions, experiments, and sequences, the states in (2) will be cleared.

_Ideally an experiment would use a final action that pushes a message to GCLD._ In lieu of this, we can set up a primitive polling loop to track running state and count the number of dispatched experiments.

In [ ]:
orch_status = requests.post(f"http://{cfg['ORCH']['host']}:{cfg['ORCH']['port']}/get_status")

dispatched_exps = set()
last_exp_count = 0
while orch_status.json()["loop_state"] == "started":
    sleep(2)
    orch_status = requests.post(f"http://{cfg['ORCH']['host']}:{cfg['ORCH']['port']}/get_status")
    active_dict = orch_status.json()['active_dict']
    for act_uuid, act_dict in active_dict.items():
        dispatched_exps.add(act_dict['act']['experiment_uuid'])
    if len(dispatched_exps) != last_exp_count:
        last_exp_count = len(dispatched_exps)
        print(last_exp_count, "experiments have been dispatched.")
print("Orch has stopped.")

In [ ]:
# use robotic sampler 'PAL' action server as placeholder for sample database server
acq_resp = requests.post(f"http://{cfg['PAL']['host']}:{cfg['PAL']['port']}/get_measured", json={"start_idx": 0})
acq_resp.status_code == 200

In [ ]:
acq_resp.json()

In [ ]:
# acquire new batch of 5 random comps
random.seed(0)
batch2_inds = random.sample(range(len(comp_space)), 10)[5:]
batch2_inds

In [ ]:
# original seed indices
comp_inds

In [ ]:
# create sequence object for holding experiments
sequence = Sequence(sequence_name='seed_sequence')

# populate sequence's experiment list
for i in batch2_inds:
    sequence.experiment_plan_list.append(
        ExperimentTemplate(
            experiment_name="SIM_measure_CP",
            experiment_params={
                "solution_ph": 13,
                "elements": ["Ni", "Fe", "La", "Ce", "Co", "Ta"],
                "element_fracs": comp_space[i],
            },
        )
    )

In [ ]:
# send sequence to Orchestrator
seq_req = requests.post(f"http://{cfg['ORCH']['host']}:{cfg['ORCH']['port']}/append_sequence", json={"sequence": sequence.as_dict()})
seq_req.status_code == 200  # successful post request

In [ ]:
# start Orch (begin or resume dispatching sequence/experiment/action queues)
orch_start = requests.post(f"http://{cfg['ORCH']['host']}:{cfg['ORCH']['port']}/start")
orch_start.status_code == 200

In [ ]:
orch_status = requests.post(f"http://{cfg['ORCH']['host']}:{cfg['ORCH']['port']}/get_status")

dispatched_exps = set()
last_exp_count = 0
while orch_status.json()["loop_state"] == "started":
    sleep(2)
    orch_status = requests.post(f"http://{cfg['ORCH']['host']}:{cfg['ORCH']['port']}/get_status")
    active_dict = orch_status.json()['active_dict']
    for act_uuid, act_dict in active_dict.items():
        dispatched_exps.add(act_dict['act']['experiment_uuid'])
    if len(dispatched_exps) != last_exp_count:
        last_exp_count = len(dispatched_exps)
        print(last_exp_count, "experiments have been dispatched.")
print("Orch has stopped.")

In [ ]:
# change 'start_idx' query parameter to slice list of measured space
start_idx = 5
acq2_resp = requests.post(f"http://{cfg['PAL']['host']}:{cfg['PAL']['port']}/get_measured?start_idx={start_idx}")
acq2_resp.status_code == 200

In [ ]:
acq2_resp.json()

In [ ]:
# further sequences/experiments on the same space/plate will aggregate results on the PAL server
# loading a new space (i.e. issuing a new experiment with different elements+pH from previous) will reset the aggregated results
# results should be queried prior to changing plates

# the following request manually resets the list of acquired samples on the PAL server
reset_resp = requests.post(f"http://{cfg['PAL']['host']}:{cfg['PAL']['port']}/clear_measured")
reset_resp.status_code == 200